In [23]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import nibabel as nib
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
ref=nib.load('/data/temp/dcm2niix_image.nii')
ref1=nib.load('/data/temp/dataconvert_image.nii')

In [32]:
flips=np.sign(ref.affine)
im1=ref.as_reoriented([[0,flips[0,0]],[1,flips[1,1]],[2,flips[2,2]]])
flips=np.sign(ref1.affine)
im2=ref1.as_reoriented([[0,flips[0,0]],[1,flips[1,1]],[2,flips[2,2]]])
#print(ref.affine[2][2])
#print(ref.as_reoriented())

In [35]:
im1.to_filename('/data/temp/dcm2niix_image_unflip.nii')
im2.to_filename('/data/temp/dataconvert_image_unflip.nii')

In [34]:
print(im2)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (512, 512, 166)
affine: 
[[   0.67773438    0.            0.         -190.16113281]
 [   0.            0.67773438    0.          -18.16113281]
 [   0.            0.            2.          325.        ]
 [   0.            0.            0.            1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 512 512 166   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.        0.6777344 0.6777344 2.        0.        0.        0.
 0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.

In [21]:
def get_slice(r):
    if isinstance(r[0],list):
        return slice(r[0][0],r[0][1]),slice(r[1][0],r[1][1]),slice(r[2][0],r[2][1])
    else:
        return slice(0,r[0]),slice(0,r[1]),slice(0,r[2])

def subimage2image(sub_image,ref_image,json_header,is_mask=True, overlay=False):
    sub,ref,js=sub_image,ref_image,json_header
    #debug
    #sub.to_filename('/data/temp/sub.nii')
    #ref.to_filename('/data/temp/ref')
    print(js)
    
    #1. Create a resampled copy of the reference image.   
    d=np.zeros(js['scaled_src_dims'])
    #2. Init source and target ranges
    ssrc=get_slice(js['scaled_src_range'])
    starg=get_slice(js['targ_range'])
    
    d[ssrc]=sub.get_fdata()[starg]
    ref_sampled_im=nib.Nifti1Image(d,np.diag(np.sign(np.diagonal(ref_image.affine))))
    #debug
    #ref_sampled_im.to_filename('/data/temp/ref_sampled_im')    
    pd=ref._header['pixdim']
    
    #interpolation order
    order=0 if is_mask else 3
    res=nibabel.processing.conform(ref_sampled_im,ref.shape,voxel_size=(pd[1],pd[2],pd[3]),order=order)
    res.set_sform(ref.affine)
    #print(res.affine)
    return res


In [22]:
res=subimage2image(subim,ref,hdr)
res.to_filename('/data/TMORPH/I3CR/MW101_MR_tx1_GK/test/out_Tum1_LFront_orig_space.nii')


{'scaled_src_dims': [257, 257, 159], 'targ_range': [[0, 70], [0, 70], [0, 70]], 'scaled_src_range': [[48, 118], [85, 155], [67, 137]]}
[[-1.00000000e+00 -2.05103490e-10  0.00000000e+00  1.25288139e+02]
 [ 2.05103490e-10 -1.00000000e+00  0.00000000e+00  1.65966095e+02]
 [ 0.00000000e+00  0.00000000e+00  2.50000000e+00 -5.56991539e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
